In [1]:
# import requests
# import pandas as pd
import zp_tools as zpt

In [4]:
# Configuration
team_id = "2740" # CRYO-GEN

*** Now lets get the team data, this returns a pandas dataframe.

In [8]:
team_data = zpt.get_team_data(2740)
team_data.head(10)

,div,divw,flag,r,ftp,name,aid,age,zada,w,...,skill_seg,skill_power,zwid,status,reg,email,h_1200_watts,h_1200_wkg,h_15_watts,h_15_wkg
0,0,0,us,0,"[0, 0]",Adrienne Tyler,0,0,0,"[0, 0]",...,0,0,1046124,,0,,NaN,NaN,NaN,NaN
1,30,30,pl,5,"[242, 0]",Agata Kakiel CRYO-GEN,0,33,0,"[79.0, 0]",...,1065,1884,1100063,,1,,254,3.2,430,5.4
2,40,0,us,10,"[202, 0]",Ahmad H,0,35,0,"[86.2, 0]",...,346,1564,840295,,1,,212,2.5,483,5.6
3,40,0,gb-eng,43,"[213, 0]",Anthony Tyler Cryo - Gen,4259267,50,0,"[89.4, 0]",...,624,1727,325493,,1,,224,2.5,515,5.8
4,20,0,kw,44,"[253, 0]",Aziz Al-Khamis (CRYO-GEN Kestrels),0,33,0,"[74.0, 0]",...,2129,2301,1600606,,1,,266,3.6,663,9.0
5,40,40,us,7,"[126, 0]",Bari Berger CRYO-GEN/ HERD 7209,3680526,51,0,"[81.0, 0]",...,60,999,1607012,,1,,132,1.6,300,3.7
6,30,0,ca,13,"[290, 0]",Bartlomiej Sanak,6392311,30,0,"[90.0, 0]",...,1224,2273,1211455,,1,,305,3.4,740,8.2
7,40,0,us,11,"[215, 0]",Brian Bergman,0,56,0,"[102.1, 0]",...,998,1559,1609882,,1,,226,2.2,517,5.1
8,30,0,gb-sct,18,"[212, 0]",Chris Bruce,0,36,0,"[74.0, 0]",...,1609,1947,1336673,,1,,223,3.0,631,8.5
9,30,0,us,2,"[202, 0]","Chris Kotowicz (Cryo-Gen, TBR)",0,48,0,"[73.9, 0]",...,0,2020,567558,,1,,212,2.9,610,8.3
